In [23]:
%%bash

# gsutil cp -r gs://otar007-cosmic/21.04/CTTV007-10-03-2021.json.gz \
#     ~/project_data/ot/evidence_input/21.04/cosmic/

# gsutil cp -r gs://otar007-cosmic/archives/CTTV007-12-08-2020.json.gz  \
#     ~/project_data/ot/evidence_input/21.04/cosmic/
             
gsutil cp -r gs://otar007-cosmic/21.04/CTTV007-16-03-2021.json.gz  \
    ~/project_data/ot/evidence_input/21.04/cosmic/


Copying gs://otar007-cosmic/21.04/CTTV007-16-03-2021.json.gz...
- [1 files][  1.7 MiB/  1.7 MiB]                                                
Operation completed over 1 objects/1.7 MiB.                                      


In [11]:
import pyspark.sql
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext
import re

global spark

# SparkContext.setSystemProperty('spark.executor.memory', '20g')

spark = (pyspark.sql.SparkSession
    .builder
    .appName("phenodigm_parser")
    .config("spark.executor.memory", '10g')
     .config("spark.driver.bindAddress", "localhost")
    .config("spark.driver.memory", '10g')
    .getOrCreate()
)

#   


print('Spark version: ', spark.version)
cosmicFile = '/Users/dsuveges/project_data/ot/evidence_input/21.04/cosmic/CTTV007-10-03-2021.json.gz'
cosmicDf = (
    spark.read.json(cosmicFile)
    .toPandas()
)


cosmicDf.show()

Spark version:  3.0.0


Py4JJavaError: An error occurred while calling o85.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 8, c02zq14flvdn, executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2023)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:1972)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:1971)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:1971)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(DAGS

In [12]:
import pandas as pd




In [19]:
cosmic_pdf = pd.read_json(cosmicFile, compression='infer', lines=True, dtype=False)
cosmic_pdf.diseaseFromSourceMappedId = cosmic_pdf.diseaseFromSourceMappedId.apply(lambda x: x.split('/')[-1])
cosmic_pdf.drop(['studyId','literature'], axis=1, inplace=True)
cosmic_pdf.to_json('cosmic_updated.json', orient='records', lines=True)

In [16]:
{
    'mutatedSamples': [
        {'numberMutatedSamples': 29, 
         'numberSamplesTested': 29, 
         'numberSamplesWithMutationType': 29, 
         'functionalConsequenceId': 'SO_0001565'
        }
    ], 
    'resourceScore': 1.0, 
    'targetFromSourceId': 'ENSG00000171094', 
    'diseaseFromSourceMappedId': 'UBERON_0012429', 
    'datasourceId': 'cancer_gene_census', 
    'datatypeId': 'somatic_mutation'
}